<a href="https://colab.research.google.com/github/yatharthmahesh/Machine-Learning/blob/master/fruits_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install kaggle

In [49]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"yatharthmahesh","key":"14f9763c7f8aa9e9253323c69c7e0b49"}
with open("/content/.kaggle/kaggle.json", 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [52]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [53]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [55]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
unanimad/dataisbeautiful                                    Reddit - Data is Beautiful                           11MB  2020-03-21 22:28:28            593  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)  646MB  2020-03-20 23:31:34          25249  
rubenssjr/brasilian-houses-to-rent                          brazilian_houses_to_rent                            117KB  2020-03-15 01:12:22            592  
sudalairajkumar/novel-corona-virus-2019-dataset             Novel Corona Virus 2019 Dataset                     389KB  2020-03-23 05:12:36          73235  
kimjihoo/coronavirusdataset                                 Data

In [56]:
!kaggle datasets download -d moltean/fruits -p/content

fruits.zip: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
!unzip \*.zip

Archive:  fruits.zip
replace fruits-360_dataset/fruits-360/LICENSE? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [58]:
import os
train_dir="/content/fruits-360_dataset/fruits-360/Training/"
test_dir="/content/fruits-360_dataset/fruits-360/Test/"
z=os.listdir(train_dir)
z=['Lemon','Tomato Cherry Red','Banana','Mango','Orange','Walnut','Raspberry','Pineapple','Papaya']
print(z)

['Lemon', 'Tomato Cherry Red', 'Banana', 'Mango', 'Orange', 'Walnut', 'Raspberry', 'Pineapple', 'Papaya']


In [0]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Dense,Flatten
import tensorflow as tf

In [0]:
from keras_preprocessing.image import ImageDataGenerator

In [61]:
train1=ImageDataGenerator(rescale=1/255)
train_gen=train1.flow_from_directory(train_dir,class_mode='categorical',classes=z,batch_size=36,target_size=(100,100))
train2=ImageDataGenerator(rescale=1/255)
test_gen=train2.flow_from_directory(test_dir,class_mode='categorical',classes=z,batch_size=36,target_size=(100,100))

Found 4650 images belonging to 9 classes.
Found 1565 images belonging to 9 classes.


In [62]:
model=tf.keras.models.Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(100,100,3)),
                               MaxPooling2D(2,2),
                               Dropout(0.2),
                               Conv2D(64,(3,3),activation='relu'),
                               MaxPooling2D(2,2),
                               Dropout(0.2),
                               Flatten(),
                               Dense(128,activation='relu'),
                               Dense(9,activation='softmax')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 33856)            

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [64]:
model.fit_generator(train_gen,validation_data=test_gen,epochs=15)

Epoch 1/15
130/130 [==============================] - 6s 47ms/step - loss: 0.1929 - acc: 0.9497 - val_loss: 0.0507 - val_acc: 0.9738
Epoch 2/15
130/130 [==============================] - 5s 41ms/step - loss: 8.8045e-05 - acc: 1.0000 - val_loss: 0.0358 - val_acc: 0.9853
Epoch 3/15
130/130 [==============================] - 5s 41ms/step - loss: 2.6660e-05 - acc: 1.0000 - val_loss: 0.0080 - val_acc: 1.0000
Epoch 4/15
130/130 [==============================] - 5s 41ms/step - loss: 9.0163e-06 - acc: 1.0000 - val_loss: 0.0093 - val_acc: 0.9949
Epoch 5/15
130/130 [==============================] - 5s 42ms/step - loss: 5.3812e-06 - acc: 1.0000 - val_loss: 0.0142 - val_acc: 0.9917
Epoch 6/15
130/130 [==============================] - 5s 41ms/step - loss: 2.6960e-06 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 0.9942
Epoch 7/15
130/130 [==============================] - 5s 41ms/step - loss: 1.6423e-06 - acc: 1.0000 - val_loss: 0.0086 - val_acc: 0.9923
Epoch 8/15
130/130 [=========================

In [47]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(100, 100))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes)
  for i in range(9):
    if(classes[0][i]==1):
      print(z[i])
  

Saving mango.jfif to mango.jfif
[[0. 0. 1. 0. 0. 0. 0. 0. 0.]]
Banana
